In [46]:
# !pip install flair beautifulsoup4 pyinterval torchcontrib

In [18]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from flair.data import Sentence 
from flair.models import SequenceTagger
from tqdm.auto import tqdm
from segtok.segmenter import split_single
import logging
import numpy as np
import ast
from flair.data import Corpus
from flair.datasets import ColumnCorpus
from flair.embeddings import WordEmbeddings, StackedEmbeddings, FlairEmbeddings
from flair.trainers import ModelTrainer
import torch

# SCRAPE PAGES

In [80]:
def build_dataset(url_df, page_limit=None):

    urls = url_df.values.squeeze().tolist()[:page_limit]
    pages = []
    
    for url in urls:

        try:

            data = requests.get(url)
            soup = BeautifulSoup(data.content, 'html.parser')
            tags = soup.find_all()

            text = []
            blacklist = [
                '[document]',
                'noscript',
                'header',
                'html',
                'meta',
                'head',
                'input',
                'script',
                'style',
                'title'
            ]

            for t in tags:
                if t.name not in blacklist and t.parent.name not in blacklist:
                    clean_text = " ".join(t.text.split())
                    if len(clean_text) > 1:
                        text.append(clean_text)

            pages.append(text)

        except Exception as e:
            logging.exception(str(e))
            pages.append(np.nan)

    df = pd.DataFrame(columns=['raw_text'])
    df['raw_text'] = pages
    # df = df.replace(np.nan, '', regex=True)
    return df

url_df = pd.read_csv('furniture stores pages.csv')
pages_df = build_dataset(url_df)#, page_limit=5)
pages = pages_df['raw_text'].values.tolist()

print(pages_df.shape)
print(url_df.shape)

del_lis = []
for i in tqdm(range(len(pages)), desc='Pages Progress'):
    if type(pages[i]) == str and len(pages[i]) < 10:
        del_lis.append(i) 

nans = pages_df.loc[pd.isna(pages_df["raw_text"]), :].index.tolist()
del_lis = np.concatenate((del_lis, nans))

pages_df.drop(del_lis, inplace=True)                         
pages = pages_df['raw_text'].values.tolist()
url_df.drop(del_lis, inplace=True)                         

# pages_df.drop([30], inplace=True)                         
# pages = pages_df['raw_text'].values.tolist()
# url_df.drop([30], inplace=True)                         

# # pages_df.drop(['Unnamed: 0'], axis=1, inplace=True)
# pages_df.reset_index(drop=True, inplace=True)
# url_df.reset_index(drop=True, inplace=True)

pages_df.to_csv('pages_content.csv')
url_df.to_csv('furniture stores pages.csv')

print(pages_df.shape)
print(url_df.shape)

ERROR:root:HTTPSConnectionPool(host='furnish123watertown.com', port=443): Max retries exceeded with url: /products/ (Caused by SSLError(SSLCertVerificationError("hostname 'furnish123watertown.com' doesn't match either of '*.wpengine.com', 'wpengine.com'")))
Traceback (most recent call last):
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9/site-packages/urllib3/connectionpool.py", line 699, in urlopen
    httplib_response = self._make_request(
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9/site-packages/urllib3/connectionpool.py", line 382, in _make_request
    self._validate_conn(conn)
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9/site-packages/urllib3/connectionpool.py", line 1010, in _validate_conn
    conn.connect()
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9/site-packages/urllib3/connection.py", line 469, in connect
    _match_hostname(cert, self.assert_hostname or server_hostname)
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/p

ERROR:root:HTTPSConnectionPool(host='houseofhollingsworthblog.com', port=443): Max retries exceeded with url: /products/locker (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f17280ccc40>: Failed to establish a new connection: [Errno 110] Connection timed out'))
Traceback (most recent call last):
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9/site-packages/urllib3/util/connection.py", line 96, in create_connection
    raise err
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9/site-packages/urllib3/util/connection.py", line 86, in create_connection
    sock.connect(sa)
TimeoutError: [Errno 110] Connection timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9

ERROR:root:('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Traceback (most recent call last):
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9/site-packages/urllib3/connectionpool.py", line 699, in urlopen
    httplib_response = self._make_request(
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9/site-packages/urllib3/connectionpool.py", line 445, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9/site-packages/urllib3/connectionpool.py", line 440, in _make_request
    httplib_response = conn.getresponse()
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9/http/client.py", line 1371, in getresponse
    response.begin()
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9/http/client.py", line 319, in begin
    version, status, reason = self._read_status()
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9/ht

ERROR:root:HTTPSConnectionPool(host='www.jandjtreasuretrove.net', port=443): Max retries exceeded with url: /apps/webstore/products/show/8098794 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f172409ca90>: Failed to establish a new connection: [Errno -2] Name or service not known'))
Traceback (most recent call last):
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9/site-packages/urllib3/util/connection.py", line 73, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9/socket.py", line 954, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -2] Name or service not known

During handling of the above exception, another excep

ERROR:root:('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Traceback (most recent call last):
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9/site-packages/urllib3/connectionpool.py", line 699, in urlopen
    httplib_response = self._make_request(
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9/site-packages/urllib3/connectionpool.py", line 445, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9/site-packages/urllib3/connectionpool.py", line 440, in _make_request
    httplib_response = conn.getresponse()
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9/http/client.py", line 1371, in getresponse
    response.begin()
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9/http/client.py", line 319, in begin
    version, status, reason = self._read_status()
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9/ht

...

ERROR:root:HTTPSConnectionPool(host='divine-interiors.co.uk', port=443): Max retries exceeded with url: /products/sofa-deposit (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f1723d1f700>: Failed to establish a new connection: [Errno -2] Name or service not known'))
Traceback (most recent call last):
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9/site-packages/urllib3/util/connection.py", line 73, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9/socket.py", line 954, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -2] Name or service not known

During handling of the above exception, another exception occurred:

Tr

ERROR:root:HTTPSConnectionPool(host='www.harperandfaith.co.uk', port=443): Max retries exceeded with url: /products/jupiter-bed (Caused by SSLError(SSLCertVerificationError("hostname 'www.harperandfaith.co.uk' doesn't match either of 'myshopify.com', '*.myshopify.com'")))
Traceback (most recent call last):
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9/site-packages/urllib3/connectionpool.py", line 699, in urlopen
    httplib_response = self._make_request(
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9/site-packages/urllib3/connectionpool.py", line 382, in _make_request
    self._validate_conn(conn)
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9/site-packages/urllib3/connectionpool.py", line 1010, in _validate_conn
    conn.connect()
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9/site-packages/urllib3/connection.py", line 469, in connect
    _match_hostname(cert, self.assert_hostname or server_hostname)
  File "/home/gabriel/miniconda3/en

ERROR:root:HTTPSConnectionPool(host='jopa-interiors.co.uk', port=443): Max retries exceeded with url: /products/black-weave-cushion-with-tag (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f17231ff2e0>: Failed to establish a new connection: [Errno -2] Name or service not known'))
Traceback (most recent call last):
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9/site-packages/urllib3/util/connection.py", line 73, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9/socket.py", line 954, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -2] Name or service not known

During handling of the above exception, another exception

ERROR:root:HTTPSConnectionPool(host='www.furlon.co.uk', port=443): Max retries exceeded with url: /products/194 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f17236e6dc0>: Failed to establish a new connection: [Errno -2] Name or service not known'))
Traceback (most recent call last):
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9/site-packages/urllib3/util/connection.py", line 73, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9/socket.py", line 954, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -2] Name or service not known

During handling of the above exception, another exception occurred:

Traceback (most r

ERROR:root:('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
Traceback (most recent call last):
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9/site-packages/urllib3/connectionpool.py", line 699, in urlopen
    httplib_response = self._make_request(
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9/site-packages/urllib3/connectionpool.py", line 382, in _make_request
    self._validate_conn(conn)
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9/site-packages/urllib3/connectionpool.py", line 1010, in _validate_conn
    conn.connect()
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9/site-packages/urllib3/connection.py", line 416, in connect
    self.sock = ssl_wrap_socket(
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9/site-packages/urllib3/util/ssl_.py", line 449, in ssl_wrap_socket
    ssl_sock = _ssl_wrap_socket_impl(
  File "/home/gabriel/miniconda3/envs/pt_nlp/lib/python3.9/site-packages/urllib3/util/ssl

# READ PRODUCT ANNOTATIONS

In [175]:
with open("products.txt", "r") as f:
    lines = f.readlines()
prods=[[]]
# skip = False

for line in lines:
    line = line[:-1]
#     if line == 'X':
#         skip = True
#         deleted = len(prods) - 1 # index where to delete url and html
    if line == '':
#         if skip:
#             skip = False
#         else:
        prods.append([])
    elif line != '_':
        prods[-1].append(line)
    
print(len(prods))

truths = []
for i in range(len(prods)):
    if prods[i] != []:
        truth = False
        for sent in ast.literal_eval(pages_df['raw_text'].iloc[i]):
            for prod in prods[i]:
                if prod.lower() in sent.lower():
                    truth = True
                    break
            
        truths.append(truth)
                
    else:
        truths.append(True)

        
print(truths)


print(len(truths))
ti = [i for i,x in enumerate(truths) if x == False]
print(len(ti))

['Factory Buys Euro Top Mattress - King', 'Neo Bed Frame Fabric - Charcoal Queen', 'Wooden Bed Frame Timber Base Bedroom Kids - Double', 'Factory Buys Euro Top Mattress - Double', 'Tiyo Bed Frame PU Leather Gas Lift Storage - White Queen', 'Cole LED Bed Frame Leather Gas Lift Storage Cole - Black Queen', 'Set of 2 Bedside Table Nightstands 2 Drawers Storage Cabinet Bedroom Side White', 'Factory Buys Euro Top Mattress - Queen', 'DreamZ 35CM Thickness Euro Top Egg Crate Foam Mattress - King', '6 Chest of Drawers Tallboy Dresser Table Storage Cabinet Oak Bedroom', 'LED RGB Bedside Tables Drawers High Gloss Nightstand Black', 'Bedside Tables 3 Drawers LED High Gloss White', 'Cole LED Bed Frame PU Leather Gas Lift Storage - White Double', 'Tallboy 6 Drawers Storage Cabinet White', 'Nino Bed Frame PU Leather - Black Queen', 'Bedside Tables Side Table RGB LED Lamp 3 Drawers Nightstand Gloss Black', 'Lumi LED Bed Frame Leather Gas Lift Storage - Black Queen', 'Lumi LED Bed Frame PU Leather Gas

# CREATE FLAIR DATASET

In [384]:
from interval import interval

def write_ds(pages, prods, f):
    
    pages = [ast.literal_eval(page) for page in pages]
    
    prods_idxs = []
    for idx in range(len(prods)):
        prods_idxs.append([])
        
        for sid, sent in enumerate(pages[idx]):
            sent = sent.lower()
            prods_idxs[idx].append([])
            
            for prod in prods[idx]:
                prod = prod.lower()
                
                if prod in sent:
                    prods_idxs[idx][sid].extend([
                        [m.start(), m.end()]
                        for m in re.finditer(prod, sent.strip())
                    ])
            prods_idxs[idx][sid] = interval(*sorted(prods_idxs[idx][sid], key=lambda tup: tup[0]))

    for idx in tqdm(list(range(len(prods)))):
        for sid, sent in enumerate(pages[idx]):
            sent = Sentence(sent, use_tokenizer=True)

            if not len(sent):
                continue
                
            isChunk = False
            
            for word in sent[:min(len(sent), 512)]:
                f.write(word.text)
                
                isProd = False
                for prod_int in prods_idxs[idx][sid].components:
                    if interval[word.start_position, word.end_position] in prod_int:
                        isProd = True
                        if isChunk:
                            f.write(f' I-PRODUCT\n')
                            break
                        else:
                            f.write(f' B-PRODUCT\n')
                            isChunk = True
                            break
                if not isProd:
                    f.write(f' O\n')
            f.write(f'\n')
    
### total = 123
tr_idx = 103
ts_idx = 113
dv_idx = 123

with open('column_product_ds/train.txt', 'w') as f:
    write_ds(pages_df['raw_text'].values.tolist()[:tr_idx], prods[:tr_idx], f)

with open('column_product_ds/test.txt', 'w') as f:
    write_ds(pages_df['raw_text'].values.tolist()[tr_idx:ts_idx], prods[tr_idx:ts_idx], f)

with open('column_product_ds/dev.txt', 'w') as f:
    write_ds(pages_df['raw_text'].values.tolist()[ts_idx:dv_idx], prods[ts_idx:dv_idx], f)

  0%|          | 0/103 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

# FINE-TUNE MODEL

In [92]:
# 1. load the corpus (Ontonotes does not ship with Flair, you need to download and reformat into a column format yourself)
columns = {0: "text", 1: "ner"}
corpus: Corpus = ColumnCorpus(
                "column_product_ds", columns,
                train_file='train.txt',
                test_file='test.txt',
                dev_file='dev.txt'
#                 tag_to_bioes="ner"
            )

# 2. what tag do we want to predict?
tag_type = 'ner'

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)

# 4. initialize fine-tuneable transformer embeddings WITH document context
# from flair.embeddings import TransformerWordEmbeddings

# embeddings = TransformerWordEmbeddings(
#     model='bert-base-uncased', #'xlm-roberta-large',
#     layers="-1",
#     subtoken_pooling="first",
#     fine_tune=False,
#     use_context=True,
# )

# 5. initialize bare-bones sequence tagger (no CRF, no RNN, no reprojection)
# from flair.models import SequenceTagger

# tagger = SequenceTagger(
#     hidden_size=256,
#     embeddings=embeddings,
#     tag_dictionary=tag_dictionary,
#     tag_type='ner',
# #     use_crf=False,
# #     use_rnn=False,
# #     reproject_embeddings=False,
# )
# tagger = SequenceTagger.load('ner-ontonotes-fast')  # ner-ontonotes-large
tagger.embeddings.list_embedding_1.lm.document_delimiter = "\n"
tagger.embeddings.list_embedding_2.lm.document_delimiter = "\n"
# 6. initialize trainer
trainer = ModelTrainer(tagger, corpus)  # , optimizer=torch.optim.AdamW)

# 7. run training with XLM parameters (20 epochs, small LR)
from torch.optim.lr_scheduler import OneCycleLR

trainer.train('ner-fast-finetune-final',
#               train_with_dev=True,
              train_with_test=True,
              save_final_model=True,
              learning_rate=5.0e-6,
              mini_batch_size=32,
              mini_batch_chunk_size=1,
              max_epochs=1,
#               scheduler=OneCycleLR,
              embeddings_storage_mode='none',
              use_swa=True,
              weight_decay=0.,
              )
tagger.save('ner-fast-finetuned_model.pt')

2022-01-30 18:56:26,886 Reading data from column_product_ds
2022-01-30 18:56:26,887 Train: column_product_ds/train.txt
2022-01-30 18:56:26,888 Dev: column_product_ds/dev.txt
2022-01-30 18:56:26,888 Test: column_product_ds/test.txt


/tmp/ipykernel_12997/68817071.py:15: DeprecationWarning: Call to deprecated method make_tag_dictionary. (Use 'make_label_dictionary' instead.) -- Deprecated since version 0.8.
  tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)


2022-01-30 18:56:37,725 ----------------------------------------------------------------------------------------------------
2022-01-30 18:56:37,726 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings('en-crawl')
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.25, inplace=False)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 1024)
        (decoder): Linear(in_features=1024, out_features=275, bias=True)
      )
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.25, inplace=False)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 1024)
        (decoder): Linear(in_features=1024, out_features=275, bias=True)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=2348, out_features=2348, bias=True)
  (rnn): LSTM(2348, 256, batch_first=True

{'test_score': 0,
 'dev_score_history': [0.0],
 'train_loss_history': [0.16393901091856575],
 'dev_loss_history': [tensor(0.1145, device='cuda:0')]}

# TEST MODEL

In [110]:
tagger_roberta = SequenceTagger.load('ner-ontonotes-large')  
tagger_finetuned = SequenceTagger.load('ner-fast-finetuned_model.pt')

def batch(iterable, n=16):
    l = len(iterable)
    for ndx in tqdm(range(0, l // 3, n), leave=False, desc='Sentences Progress'):
        yield iterable[ndx:min(ndx + n, l)]

pages = pd.read_csv('pages_content.csv')['raw_text'].values.tolist()
urls = pd.read_csv('furniture stores pages.csv').values.tolist()

df_ents = []

for pid in tqdm(list(range(len(pages))), desc='Pages Progress'):
    page = ast.literal_eval(pages[pid])
    sentences = [Sentence(sent[:min(len(sent), 512)], use_tokenizer=True) for line in page for sent in split_single(line)
                 if sent != '' and sent != ' ' and sent != '\n' and sent != '\t']
    
    for sent_batch in batch(sentences):        
        tagger_finetuned.predict(sent_batch)
        tagger_roberta.predict(sent_batch)

    entities = []
    for sent in sentences:
        for entity in sent.get_spans('ner'):
            if entity.tag == 'PRODUCT' and entity.score > 0.75:
                if not any(substring in entity.to_plain_string().lower() for substring in ['google', 'safari', 'apple', 'ipad', 'iphone']):
                    if len(entity.to_plain_string()) < 40 and len(entity.to_plain_string()) > 6:
                        print(entity)
                        entities.append(entity.to_plain_string())
    df_ents.append(entities)   
        
pd.DataFrame({'url': urls, 'product': df_ents}).to_csv('EXTRACTED_PRODS.csv') 

Pages Progress:   0%|          | 0/614 [00:00<?, ?it/s]

Sentences Progress:   0%|          | 0/20 [00:00<?, ?it/s]

Sentences Progress:   0%|          | 0/12 [00:00<?, ?it/s]

Span [3,4,5,6]: "Cirrus LED Reading Light"   [− Labels: PRODUCT (0.9977)]
Span [12]: "Beadlight"   [− Labels: PRODUCT (0.9097)]


Sentences Progress:   0%|          | 0/20 [00:00<?, ?it/s]

...

Sentences Progress:   0%|          | 0/17 [00:00<?, ?it/s]

Span [1,2,3]: "JAHBK - King"   [− Labels: PRODUCT (0.9955)]
Span [9,10,11]: "JAHBQ - Queen"   [− Labels: PRODUCT (0.9982)]


Sentences Progress:   0%|          | 0/1 [00:00<?, ?it/s]

Sentences Progress:   0%|          | 0/5 [00:00<?, ?it/s]

Sentences Progress:   0%|          | 0/9 [00:00<?, ?it/s]